<a href="https://colab.research.google.com/github/Amankumar014/automation_in_linkedin/blob/main/automated_linkedin_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# .........Creating an automated system that monitors competitors' LinkedIn activity

## Importing useful pakages

In [1]:
!pip install linkedin_api

In [2]:
from linkedin_api import Linkedin
from linkedin_api.client import ChallengeException
from bs4 import BeautifulSoup
import requests


## Step 1: Monitoring Competitors' Activity

In [3]:
def monitor_competitors():
    competitor_profile_link = [
        'https://www.linkedin.com/in/rsashikumar/',
        'https://www.linkedin.com/in/nitin-22by7/',
        'https://www.linkedin.com/in/rajesh-kumar-314264a9/',
    ]

    new_connections = []

    for profile_link in competitor_profile_link:
        # getting the urls of the competitors profile and storing it in the response variable.
        response = requests.get(profile_link)

        if response.status_code == 200 or response.status_code == 201:  # 200 and 201 means the code runned successfully.
            soup = BeautifulSoup(response.text, 'html.parser') # we are trying to get the html content of the response for further use .
            connections_section = soup.find('section', {'id': 'connections'})


            if connections_section:
                # Extract the list of connections
                connections = connections_section.find_all('li', {'class': 'connection-item'})

                for connection in connections:
                    # Extract the connection details (name, profile URL, etc.)
                    connection_name = connection.find('span', {'class': 'actor-name'}).text
                    connection_profile_url = connection.find('a')['href']

                    # Store the connection data in a dictionary
                    connection_data = {
                        'name': connection_name,
                        'profile_url': connection_profile_url,
                    }

                    new_connections.append(connection_data)

    return new_connections

## Step 2 : Analysing connections profile

In [4]:
def analyze_profile(profile_url):
    profile_data = {}

    # Fetch the LinkedIn profile page
    response = requests.get(profile_url)

    if response.status_code == 200 or response.status_code == 201:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extracting the 'About' section of the profile in linkedin


        about_section = soup.find('section', {'class': 'artdeco-card ember-view relative break-words pb3'})
        if about_section:
            about_text = about_section.find('span').text.strip()
            profile_data['about'] = about_text

        # Extract the job title and company name
        experience_section = soup.find('section', {'id': 'experience-section'})
        if experience_section:
            job_title_element = experience_section.find('span', {'class': 't-16 t-black t-bold'})
            company_element = experience_section.find('span', {'class': 'pv-entity__secondary-title t-14 t-black t-normal'})

            if job_title_element:
                job_title_text = job_title_element.text.strip()
                profile_data['job_title'] = job_title_text

            if company_element:
                company_text = company_element.text.strip()
                profile_data['company'] = company_text

    return profile_data

## Step 3 : Generating messages

In [5]:
def generate_message(profile_data):
    message_template = "Hi {name}, I noticed that you recently connected with one of my competitors on LinkedIn. I would love to connect with you as well and learn more about your work at {company}."

    name = profile_data.get('name')
    company = profile_data.get('company')

    if name and company:
        message_text = message_template.format(name=name, company=company)
        return message_text

    return None

## Step 4 : Sending connection request

In [6]:
def send_connection_request(linkedin_client, profile_url):
    try:
        # Fetch the LinkedIn profile ID from the URL
        profile_id_start_index = profile_url.rfind('/') + 1 # it will mark the start of the profile ID after the last / symbol
        profile_id_end_index = profile_url.rfind('?') # it will mark the end of the profile ID
        if profile_id_end_index == -1: # if profile id end index is not found i.e -1 , it is set to be the length of the url , considering that there are no query parameters
            profile_id_end_index = len(profile_url)

        profile_id = profile_url[profile_id_start_index:profile_id_end_index]

        # Send the connection request with a personalized message
        message_text = generate_message(analyze_profile(profile_url))

        if message_text:
            linkedin_client.send_invitation(
                invitation_message=message_text,
                profile_id=profile_id,
            )

            print(f"Connection request sent to {profile_url}")

    except ChallengeException as e:
        print(f"Failed to send connection request to {profile_url}: {e}")